<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import pandas as pd
import numpy as np

# Загрузка тренировочных данных данных

In [4]:
train = pd.read_parquet('data/input/data_fusion_train.parquet')
train = train[train.category_id != -1]

In [5]:
train.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,None
17,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,70,None
40,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,84,None


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6483226 entries, 1 to 45729281
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   receipt_id         int64  
 1   receipt_dayofweek  int32  
 2   receipt_time       object 
 3   item_name          object 
 4   item_quantity      float64
 5   item_price         int64  
 6   item_nds_rate      int32  
 7   category_id        int64  
 8   brands             object 
dtypes: float64(1), int32(2), int64(3), object(3)
memory usage: 445.2+ MB


# Сводные данные

In [7]:
# Общее число записей
count = train.shape[0]
count

6483226

In [8]:
# Популярные категории
print(f"Всего {train['category_id'].value_counts().count()} категорий")
np.round(train['category_id'].value_counts()/count, 6) * 100

Всего 98 категорий


203    20.1637
84     13.9206
80     10.3472
78      7.7616
71      6.5104
        ...   
106     0.0028
97      0.0017
46      0.0010
143     0.0004
121     0.0000
Name: category_id, Length: 98, dtype: float64

In [9]:
train.loc[train['category_id'] == 78, 'item_name'].value_counts()

Сметана 10                                          12042
Молоко Мудрая хозяйка 2,5% пленка 0,8 л БЗМЖ         7844
Сметана 10г, 16 ккал                                 6532
ЯЙЦО СТОЛОВОЕ 1КАТ. ПТИЦЕФАБРИКА ПИОНЕРСКОЕ 10ШТ     6358
Яйцо куриное КОЛЬЦОВСКОЕ СО 10шт                     5194
                                                    ...  
CМЕТАНА  ЛОСЕВО 250Г 15%                                1
Продукт кисломолочный Тема Био                          1
Желе  сливочно  кофейное                                1
 НЕЖНЫЙ С СОКОМ БАНАНА И КЛУБНИКИ 1.2% 100Г             1
Сырок творожный с ванилью,шт                            1
Name: item_name, Length: 2870, dtype: int64

In [10]:
# Популярные Бренды
print(f"Всего {train['brands'].value_counts().count()} Брендов")
brands_serias = np.round(train['brands'].value_counts()/count, 6) * 100
brands_serias

Всего 2624 Брендов


кока-кола      0.8274
орбит          0.4102
пепси          0.2143
пепси-кола     0.1690
вискас         0.1414
                ...  
pulsar         0.0000
tnb            0.0000
cerave         0.0000
росток         0.0000
азбука моря    0.0000
Name: brands, Length: 2624, dtype: float64

# Выделение количества и единицы измерения

In [70]:
item_name_col_df = train[['item_name']].drop_duplicates()
item_name_col_df.item_name = item_name_col_df.item_name.str.lower()
item_name_col_df

,item_name
1,"молоко 3,2%,шт"
3,"компот из изюма, 114 ккал"
4,"макаронные изделия отварные (масло сливочное),..."
17,кофе капучино большой эден 18
40,хлеб на сыворотке 350г
...,...
45669181,"напиток энерг. ред булл 0,355л"
45681543,хеменгуэй дайкири
45690702,"пиво светлое ""халзан"" 4,5 % об, пл/б. 1,5 л(шт)"
45692298,экспресс педикюр


In [71]:
reg_dict = {'gram' : '\d{1,5}.{0,1}г', 
            'kg' : '\d{1,5}.{0,1}кг', 
            'litr' : '\d{1,3}[.|,]{0,1}\d{1,3}.{0,1}л'}

In [74]:
item_name_col_df = train[['item_name']].drop_duplicates()
item_name_col_df.item_name = item_name_col_df.item_name.str.lower()
for typ in reg_dict.keys():
    ser_filtr = item_name_col_df.item_name.str.contains(reg_dict[typ], na = False)
    item_name_col_df.loc[ser_filtr, 'ed_izm'] = typ
    col = item_name_col_df.item_name.str.extract(
        f"({reg_dict[typ]})").loc[:,0]
    item_name_col_df.loc[ser_filtr, 'col'] = col
    item_name_col_df.loc[ser_filtr, 'item_name'] = item_name_col_df.loc[ser_filtr, 
                                                'item_name'].replace(regex={
                                                reg_dict[typ] : ''})
item_name_col_df.col = item_name_col_df.col.str.replace(' ', '', regex=True)
item_name_col_df.col = item_name_col_df.col.str.replace(',', '.', regex=True)
item_name_col_df.col = item_name_col_df.col.str.replace('[^0-9,]', '', regex=True)
item_name_col_df.col = pd.to_numeric(item_name_col_df.col)
# # Концеритруем все в близкое к кг
for ed_izm in ['gram']:
    item_name_col_df.loc[item_name_col_df.ed_izm == ed_izm, 'col'
                        ] = item_name_col_df.loc[item_name_col_df.ed_izm == ed_izm, 'col'] / 1000
item_name_col_df[~pd.isna(item_name_col_df.ed_izm)]

,item_name,ed_izm,col
40,хлеб на сыворотке,gram,0.350
170,"2, лоск авт колор",kg,7.000
174,вафли с топленым молоком вес. тортугалия,gram,0.001
188,сметана кубанский молочник 20% р шт,gram,0.180
207,баклажаны,gram,0.001
...,...,...,...
45657633,2426 индейка крыло локтевая часть,gram,0.001
45666799,зефир шпек дульчия р,gram,0.100
45669181,напиток энерг. ред булл,litr,355.000
45690702,"пиво светлое ""халзан"" 4,5 % об, пл/б. (шт)",litr,15.000
